# Perceptron Multi-Couches

### Aprentissage MLP et quelques exemples

In [ ]:
#!/usr/bin/env python
# -----------------------------------------------------------------------------
# Multi-layer perceptron
# Copyright (C) 2011  Nicolas P. Rougier
#
# Distributed under the terms of the BSD License.
# -----------------------------------------------------------------------------
# This is an implementation of the multi-layer perceptron with retropropagation
# learning.
# -----------------------------------------------------------------------------

import numpy as np
 
def sigmoid(x):
    ''' Sigmoid like function using tanh '''
    return np.tanh(x)
 
def dsigmoid(x):
    ''' Derivative of sigmoid above '''
    return 1.0-x**2
 
class MLP:
    ''' Multi-layer perceptron class. '''
 
    def __init__(self, *args):
        ''' Initialization of the perceptron with given sizes.  '''
 
        self.shape = args
        n = len(args)
 
        # Build layers
        self.layers = []
        # Input layer (+1 unit for bias)
        self.layers.append(np.ones(self.shape[0]+1))
        # Hidden layer(s) + output layer
        for i in range(1,n):
            self.layers.append(np.ones(self.shape[i]))
 
        # Build weights matrix (randomly between -0.25 and +0.25)
        self.weights = []
        for i in range(n-1):
            self.weights.append(np.zeros((self.layers[i].size,
                                         self.layers[i+1].size)))
 
        # dw will hold last change in weights (for momentum)
        self.dw = [0,]*len(self.weights)
 
        # Reset weights
        self.reset()
 
    def reset(self):
        ''' Reset weights '''
 
        for i in range(len(self.weights)):
            Z = np.random.random((self.layers[i].size,self.layers[i+1].size))
            self.weights[i][...] = (2*Z-1)*0.25
 
    def propagate_forward(self, data):
        ''' Propagate data from input layer to output layer. '''
 
        # Set input layer
        self.layers[0][0:-1] = data
 
        # Propagate from layer 0 to layer n-1 using sigmoid as activation function
        for i in range(1,len(self.shape)):
            # Propagate activity
            self.layers[i][...] = sigmoid(np.dot(self.layers[i-1],self.weights[i-1]))
 
        # Return output
        return self.layers[-1]
 
 
    def propagate_backward(self, target, lrate=0.1, momentum=0.1):
        ''' Back propagate error related to target using lrate. '''
 
        deltas = []
 
        # Compute error on output layer
        error = target - self.layers[-1]
        delta = error*dsigmoid(self.layers[-1])
        deltas.append(delta)
 
        # Compute error on hidden layers
        for i in range(len(self.shape)-2,0,-1):
            delta = np.dot(deltas[0],self.weights[i].T)*dsigmoid(self.layers[i])
            deltas.insert(0,delta)
            
        # Update weights
        for i in range(len(self.weights)):
            layer = np.atleast_2d(self.layers[i])
            delta = np.atleast_2d(deltas[i])
            dw = np.dot(layer.T,delta)
            self.weights[i] += lrate*dw + momentum*self.dw[i]
            self.dw[i] = dw
 
        # Return error
        return (error**2).sum()
 
 
# -----------------------------------------------------------------------------
if __name__ == '__main__':
    import matplotlib
    import matplotlib.pyplot as plt
 
    def learn(network,samples, epochs=2500, lrate=.1, momentum=0.1):
        # Train 
        for i in range(epochs):
            n = np.random.randint(samples.size)
            network.propagate_forward( samples['input'][n] )
            network.propagate_backward( samples['output'][n], lrate, momentum )
        # Test
        for i in range(samples.size):
            o = network.propagate_forward( samples['input'][i] )
            print (i, samples['input'][i], '%.2f' % o[0],)
            print ('(expected %.2f)' % samples['output'][i])
        print
 
    network = MLP(2,2,1)
    samples = np.zeros(4, dtype=[('input',  float, 2), ('output', float, 1)])
 
    # Example 1 : OR logical function
    # -------------------------------------------------------------------------
    print ("Learning the OR logical function")
    network.reset()
    samples[0] = (0,0), 0
    samples[1] = (1,0), 1
    samples[2] = (0,1), 1
    samples[3] = (1,1), 1
    learn(network, samples)
 
    # Example 2 : AND logical function
    # -------------------------------------------------------------------------
    print("Learning the AND logical function")
    network.reset()
    samples[0] = (0,0), 0
    samples[1] = (1,0), 0
    samples[2] = (0,1), 0
    samples[3] = (1,1), 1
    learn(network, samples)
 
    # Example 3 : XOR logical function
    # -------------------------------------------------------------------------
    print("Learning the XOR logical function")
    network.reset()
    samples[0] = (0,0), 0
    samples[1] = (1,0), 1
    samples[2] = (0,1), 1
    samples[3] = (1,1), 0
    learn(network, samples)
 
    # Example 4 : Learning sin(x)
    # -------------------------------------------------------------------------
    print("Learning the sin function")
    network = MLP(1,3,1)
    
    
    samples = np.zeros(500, dtype=[('x',  float, 1), ('y', float, 1)])
    samples['x'] = np.linspace(0,1,500)
    samples['y'] = np.sin(samples['x']*np.pi)
    for i in range(10000):
        n = np.random.randint(samples.size)
        network.propagate_forward(samples['x'][n])
        network.propagate_backward(samples['y'][n])
    plt.figure(figsize=(10,5))
    # Draw real function
    x,y = samples['x'],samples['y']
    plt.plot(x,y,color='b',lw=1)
    # Draw network approximated function
    for i in range(samples.shape[0]):
        y[i] = network.propagate_forward(x[i])
    plt.plot(x,y,color='r',lw=3)
    plt.axis([0,1,0,1])
    plt.show()

Learning the OR logical function
(0, array([ 0.,  0.]), '0.00')
(expected 0.00)
(1, array([ 1.,  0.]), '0.98')
(expected 1.00)
(2, array([ 0.,  1.]), '0.98')
(expected 1.00)
(3, array([ 1.,  1.]), '1.00')
(expected 1.00)

Learning the AND logical function
(0, array([ 0.,  0.]), '-0.01')
(expected 0.00)
(1, array([ 1.,  0.]), '0.01')
(expected 0.00)
(2, array([ 0.,  1.]), '0.00')
(expected 0.00)
(3, array([ 1.,  1.]), '0.92')
(expected 1.00)

Learning the XOR logical function
(0, array([ 0.,  0.]), '0.03')
(expected 0.00)
(1, array([ 1.,  0.]), '0.93')
(expected 1.00)
(2, array([ 0.,  1.]), '0.94')
(expected 1.00)
(3, array([ 1.,  1.]), '0.03')
(expected 0.00)

Learning the sin function


### Imports

In [ ]:
import numpy as np
from sklearn import *
import sklearn as sk
from matplotlib.pyplot import *
from sklearn.decomposition import PCA

import TP2_MLP as mlp

### Question 1 : Créer un ensemble de points avec bruit gaussien

In [ ]:
%matplotlib inline 


def f_sin(X) :
    Y = np.zeros((X.size, 1))
    for i in range(X.size) :
        #Y[i] = X[i]*np.sin(X[i])
        Y[i] = np.sin(X[i])
    return Y


X = np.array(range(0,20))/2
Y = np.sin(X)

Y = Y + np.random.normal(0, 0.2, 20)

figure(1)
plot(X,Y)
scatter(X,Y)
show()

### Question 2 : Apprendre avec MLP et tracer les courbes

In [ ]:
import TP2_MLP as mlp
mlp = mlp.MLP(1,3,1)

for i in range(1000):
    n = np.random.randint(X.size)
    mlp.propagate_forward(X[n])
    mlp.propagate_backward(Y[n])


figure(2)

# Points d'apprentissage
scatter(X,Y)
# Courbe réelle : Blue
plot(X,Y,color='b',lw=1)
# Courbe estimée : Red
Y_approx = np.zeros(20)
for i in range(Y.shape[0]):
    Y_approx[i] = mlp.propagate_forward(Y[i])
plot(X,Y_approx,color='r',lw=3)

show()

### Question 3 : Charger les données Iris

In [ ]:
iris = datasets.load_iris()

#print iris.data
#print iris.feature_names
#print iris.target_names
#print iris.target_names[iris.target]

print(iris.target.shape)


### Question 4 : Binariser les numéros de classes

In [ ]:
## Solution 1 : Utiliser deux neurones de sortie : 0 -> 00, 1 -> 01, 2 -> 10

target_bin_1 = np.zeros([np.size(iris.target), 2])
for i in range(iris.target.shape[0]):
    if iris.target[i] == 0 :
        target_bin_1[i][0] = 0
        target_bin_1[i][1] = 0
    elif iris.target[i] == 1 :
        target_bin_1[i][0] = 0
        target_bin_1[i][1] = 1
    elif iris.target[i] == 2 :
        target_bin_1[i][0] = 1
        target_bin_1[i][1] = 0
        
print(target_bin_1)
## Solution 2 : Utiliser trois neurones de sortie : 0 -> 100, 1 -> 010, 2 -> 001

target_bin_2 = np.zeros([np.size(iris.target), 3])
for i in range(iris.target.shape[0]):
    if iris.target[i] == 0 :
        target_bin_2[i][0] = 1
        target_bin_2[i][1] = 0
        target_bin_2[i][2] = 0
    elif iris.target[i] == 1 :
        target_bin_2[i][0] = 0
        target_bin_2[i][1] = 1
        target_bin_2[i][2] = 0
    elif iris.target[i] == 2 :
        target_bin_2[i][0] = 0
        target_bin_2[i][1] = 0
        target_bin_2[i][2] = 1
        
print(target_bin_2)

### Question 5 : MLP sur les données Iris

In [ ]:
import TP2_MLP as mlp

mlp = mlp.MLP(4,2,2)

for i in range(100):
    n = np.random.randint(iris.data.shape[0])
    mlp.propagate_forward(iris.data[n])
    mlp.propagate_backward(target_bin_1[n])


# Points d'apprentissage

IrisACP = PCA(n_components=2).fit(iris.data).transform(iris.data)

figure(33)
# Points d'apprentissage
scatter(IrisACP[:,0],IrisACP[:,1])

# Courbe estimée : Red
Iris_approx = np.zeros([150,2])

for i in range(iris.data.shape[0]):
    Iris_approx[i] = mlp.propagate_forward(iris.data[i])
    
plot(irisNorm,Iris_approx[:,1],color='r',lw=3)

show()


### Question 6 : Normaliser les données Iris 

In [ ]:
np.mean(iris.data[:,0])
np.mean(iris.data[:,1])
np.mean(iris.data[:,2])
np.mean(iris.data[:,3])

irisNorm = np.zeros(iris.data.shape)
irisNorm[:,0] = (iris.data[:,0] - np.mean(iris.data[:,0]))/np.std(iris.data[:,0])
irisNorm[:,1] = (iris.data[:,1] - np.mean(iris.data[:,1]))/np.std(iris.data[:,1])
irisNorm[:,2] = (iris.data[:,2] - np.mean(iris.data[:,2]))/np.std(iris.data[:,2])
irisNorm[:,3] = (iris.data[:,3] - np.mean(iris.data[:,3]))/np.std(iris.data[:,3])


import TP2_MLP as mlp

mlp = mlp.MLP(4,2,2)

for i in range(100):
    n = np.random.randint(irisNorm.shape[0])
    mlp.propagate_forward(irisNorm[n])
    mlp.propagate_backward(target_bin_1[n])


# Points d'apprentissage

IrisNormACP = PCA(n_components=2).fit(irisNorm).transform(irisNorm)

figure(34)
# Points d'apprentissage
scatter(IrisNormACP[:,0],IrisNormACP[:,1])

# Courbe estimée : Red
IrisNorm_approx = np.zeros([150,2])

for i in range(irisNorm.shape[0]):
    IrisNorm_approx[i] = mlp.propagate_forward(irisNorm[i])
    
plot(irisNorm,IrisNorm_approx[:,1],color='r',lw=3)

show()

### Question 7 : MLP avec la deuxième méthode de binarisation

In [ ]:
import TP2_MLP as mlp

mlp = mlp.MLP(4,2,3)

for i in range(100):
    n = np.random.randint(irisNorm.shape[0])
    mlp.propagate_forward(irisNorm[n])
    mlp.propagate_backward(target_bin_2[n])


# Points d'apprentissage

IrisNormACP = PCA(n_components=2).fit(irisNorm).transform(irisNorm)

figure(35)
# Points d'apprentissage
scatter(IrisNormACP[:,0],IrisNormACP[:,1])

# Courbe estimée : Red
IrisNorm_approx = np.zeros([150,3])

for i in range(irisNorm.shape[0]):
    IrisNorm_approx[i] = mlp.propagate_forward(irisNorm[i])
    
plot(irisNorm,IrisNorm_approx[:,1],color='r',lw=3)

show()